<a href="https://colab.research.google.com/github/deep-learning-indaba/indaba-pracs-2019/blob/master/optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical 2b:  Optimisation for Deep Learning

## Introduction

In this practical, we will take a *deep* dive into an essential part of deep learning, and machine learning in general, **optimisation**. We'll take a look at the tools that allow as to turn a random collection of weights into a state-of-the-art model for any number of applications. More specifically we'll implement a few standard optimisation algorithms for  finding the minimum of [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) and then we'll try them out on FashionMNIST.

## Learning Objectives

* Understand what optimisation algorithms are, and how they are used in the context of deep learning
* Understand gradient descent, stochastic gradient descent, and mini-batch stochastic gradient descent
* Understand the roles of batch size, learning rate and other hyper-parameters
* Implement, using TF2.0, gradient descent and a few variations of it
* Understand the strengths and weaknesses of the various optimisation algorithms covered in this practical.

In [0]:
#@title Imports (RUN ME!) { display-mode: "form" }

# TODO: Swallow output
!pip install tensorflow-gpu==2.0.0-beta0
!pip -q install pydot_ng
!pip -q install graphviz
!apt install graphviz > /dev/null

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
%matplotlib inline

display.clear_output()

print("TensorFlow executing eagerly: {}".format(tf.executing_eagerly()))

## Rosenbrock's Banana Function 🍌

In practice, when evaluating the performance of various optimisation algorithms and hyper-parameters, what we really care about is the performance on a wide range of real-world problems. However, we are easily not able to visualize what our algorithms are doing because the loss landscape for even simple neural networks trained on just about any real-world dataset will be high-dimensional. 

To solve this problem, we will use Rosenbrock's (Banana) Function as a playground for investigating how these optimisation algorithms work. The banana function is easy to visualize because it is a function that takes a 2D ($x$ and $y$) input and returns a scalar output. It is defined as:

\begin{equation}
f(x,y) = (a-x)^2+b\times(y-x^2)^2
\end{equation}

where typical values for $a$ and $b$ are $1$ and $100$, respectively. For this prac we'll use $a = 1$ and $b = 20$. The global minimum of this function is at $(a, a^2)$ or $(1, 1)$ in our case.

We can easily define this function using TensorFlow:

In [0]:
def rosenbrock_banana(x, y, a=1., b=20.):
  return tf.math.pow(a-x,2.) + b*tf.math.pow(y-tf.math.pow(x,2.),2.)

Let's try visualizing the 🍌, first using a contour plot:

In [0]:
#@title Helper functions (double click to unhide/hide the code)

def gen_2d_loss_surface(loss_func,
             n_x=100, # number of discretization points along the x-axis
             n_y=100, # number of discretization points along the x-axis
             min_x=-2., max_x=2., # extreme points in the x-axis
             min_y=-0.2, max_y=1.3 # extreme points in the y-axis
            ):
  
  # create a mesh of points at which to evaluate our function
  X,Y = np.meshgrid(np.linspace(min_x, max_x, n_x),
                    np.linspace(min_y, max_y, n_y))
  # evaluate the func at all of the points
  Z = loss_func(X, Y).numpy()
  
  return X, Y, Z
  
def make_contour_plot(X, Y, Z, levels=None):
  if levels==None:
    # generate 20 levels on a log scale
    levels = np.insert(np.logspace(0, 2.6, 20, True, base=10), 0, 0)
    
  fig = plt.figure(figsize=(9.84, 3))
  ax = fig.gca()
  
  ax.contour(X, Y, Z, levels, alpha=0.5)
  ax.contourf(X, Y, Z, levels, alpha=0.2)
  ax.set_xlabel('x')
  ax.set_ylabel('y')
  
  return fig, ax

def make_surface_plot(X, Y, Z,
                      elevation=0, azimuth_angle=0, levels=None):
  
  if levels==None:
    # generate 20 levels on a log scale
    levels = np.insert(np.logspace(0, 2.6, 20, True, base=10), 0, 0)
    
  fig = plt.figure(figsize=(10,6))
  ax = fig.gca(projection='3d')
  ax.view_init(elevation, azimuth_angle)
  ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.2)
  ax.contour(X, Y, Z, levels, cmap='viridis', alpha=0.5)
  
  ax.set_xlabel('x')
  ax.set_ylabel('y')
  
  return fig, ax


In [0]:
X, Y, Z = gen_2d_loss_surface(rosenbrock_banana)
fig, ax = make_contour_plot(X, Y, Z)

# add a marker to show the minimum
ax.plot(1, 1, 'r*', ms=20, label='minimum') 
ax.legend()

fig.show()

And now with a surface plot:

In [0]:
#@title {run: "auto"}

elevation = 39 #@param {type:"slider", min:0, max:360, step:1}
azimuth_angle = 110 #@param {type:"slider", min:0, max:360, step:1}
  
X, Y, Z = gen_2d_loss_surface(rosenbrock_banana)
fig, ax = make_surface_plot(X, Y, Z, elevation, azimuth_angle)

ax.plot([1], [1], 'r*', zs=[0], zdir='z', ms=20, label='minimum')
ax.legend()

fig.show()

As you can see, this is called the *banana* function because it contains a banana-shaped valley. Within the valley, we have a global minimum. Finding the valley is relatively easy, but finding the global minimum is difficult, which makes this a useful function for testing optimisation algorithms.

## Gradients

TODO: Steal some content from *How to build your own TF*

## Gradient Descent

Gradient descent is the most simple of learning algorithms commonly used in deep learning. However, it not only gives excellent results in many cases but also forms the basis for many other powerful optimization methods such as Momentum, RMSProp, and Adam which we will look at later in this practical. Mathematically we can describe gradient descent as follows:

\begin{equation}
\mathbf{θ}= \mathbf{θ} −\eta \times \nabla_\mathbf{θ} J(\mathbf{θ})
\end{equation}

where $\mathbf{θ}$ are the parameters of the model, $\eta$ is the learning rate, $J(\mathbf{θ})$ is the loss function, and $\nabla_\theta J(\mathbf{θ})$ is the **gradient** of the loss with respect to the parameters. What this equation tells us is that to update each of the parameters scale the gradient for each parameter by the learning rate and subtract it from the corresponding parameter, or in pseudo code:

```
for _ in range(epochs):
  grad = calc_grad(loss_func, data, params)
  params = params - learning_rate * grad
```

You might have noticed that we are sweeping a lot of critical details under the rug here! Firstly we are assuming that we can easily calculate the gradients using some `calc_grad` function, and secondly, we are ignoring the issue of batch size. 

Luckily for us, TensorFlow addresses the first detail thanks to **automatic differentiation** (AD). With AD calculating the gradients is about as simple as calling a `calc_grad` function, which means that we do not need to worry about the details of *how* to calculate the gradients. We don't need to think much about implementing derivatives for each of our operations, or the backpropagation algorithm, for example. If you want to know more about how this all works, you should check out the *Build your own TensorFlow* tutorial.

In practice, batch size is simply a hyper-parameter that we can tune. However, there are three cases that are worth knowing about:

1.   Batch size of $n$, where $n$ is the number of examples, also known as *Batch Gradient Descent*. In this case, all of the data is used to calculate the gradient at each step, which results in the most accurate estimate of the gradient and is guaranteed to converge to the global minimum, for convex optimisation surfaces, or a local minimum, for non-convex surfaces. However, this comes at the cost of not applying to online learning, where we get new examples during training.
2.   Batch size of 1, called *Stochastic Gradient Descent* (SGD). In this case, the estimate of the gradient is very noisy, and we are not gauranteed to find a minimum (local or global). However, in practice SGD, still performs very well. SGD also allows for online learning. It also turns out that having noisy estimates of the gradient acts as a form of regularisation to prevent over-fitting. Finally, because we are performing gradient descent with one example as a time, we need much less memory, which can be a significant issue when using batch gradient descent.
3.   Batch size of $m < n$, called *Mini-batch Gradient Descent*, which is a compromise between batch and stochastic gradient descent. We still have *some* noise in the gradient estimate, and we can tune the batch size to make good use of memory, but the variance of the gradient estimate is greatly reduced which leads to better convergence to local or global minima.

In deep learning, we almost always use mini-batch gradient descent. However, it is often referred to as SGD.






### Optional extra reading: choosing a batch size

You might be wondering how one chooses which batch size to use for a given problem.

One approach is to choose as big a batch size as possible. The reason we might want to do this is that a larger batch size means that our model will train faster. This speedup is because modern computer hardware, especially GPUs, is designed with parallelism in mind. In other words, by having a larger batch size, we can take better advantage of our hardware. The issue with this philosophy is that, especially with images, audio, and other multimedia data often used in deep learning, we quickly reach the memory limits of our hardware. So in practice we often choose the largest batch size that will fit into memory.

Another reason to choose a large batch size is that, [according to some research](https://arxiv.org/pdf/1803.09820.pdf), this allows you to select a higher learning rate, which in turn means that your model will train faster.

On the other hand, in practice choosing too large, a batch size can lead to training divergence, which means that we still have to tune the batch size.

You can read more about these issues in this [blog post](https://blog.janestreet.com/does-batch-size-matter/) as well as the paper linked above — the paper, in particular, discusses strategies for choosing various hyper-parameters related to optimisation in deep learning.

### Implementing SGD

SGD is incredibly simple to implement, and as you'll see later can perform very well!

In [0]:
def SGD_update(params, grads, states, hyper_params):
  # learning_rate=0.01
  # SGD doesn't have any state, however, the algorithms
  # we will look at later do!
  for param, grad in zip(params, grads):
    param.assign_sub(hyper_params['lr']*grad)

And now we can visualize each step of the SGD optimization:

(Take a look at the code and make sure that you understand it. Do you see how the code relates to the pseudo-code and equation above? For example, where do you find `calc_grad`? Once you understand the code, you should hide it again to make it easier to change the sliders and see the results.)

In [0]:
#@title Helper functions (double click to unhide/hide the code)

def optimize_banana(update_func, params, states, hyper_params):
  # plot the loss surface, minimum value and starting point
  X, Y, Z = gen_2d_loss_surface(rosenbrock_banana)
  fig, ax = make_contour_plot(X, Y, Z)
  ax.plot(1, 1, 'r*', ms=20, label='minimum') 
  ax.plot(start_x, start_y, 'b*', ms=20, label='start')

  for epoch in range(epochs):
    with tf.GradientTape() as tape:
      # we are trying to minimize is the output of the 🍌 func
      loss = rosenbrock_banana(x, y) 
    # calculate the gradients of the loss with respect to the params
    grads = tape.gradient(loss, params)

    # save the old x and y values for the plot
    old_x = params[0].numpy()
    old_y = params[1].numpy()

    # update the parameters using SGD
    update_func(params, grads, states, hyper_params)

    # plot the change in x and y for each update step
    ax.annotate('', xy=(x.numpy(), y.numpy()),
                xytext=(old_x, old_y),
              arrowprops={'arrowstyle': '->', 'color': 'k', 'lw': 1},
                   va='center', ha='center')  

  ax.plot(x.numpy(), y.numpy(), 'g*', ms=20, label='end')
  ax.legend()

  fig.show()

In [0]:
#@title Double click to unhide/hide the code {run: "auto"}
start_x = -0.4 #@param {type:"slider", min:-2, max:2, step:0.1}
start_y = 0.63334 #@param {type:"slider", min:-0.26666, max:1.0666, step:0.1}
learning_rate = 0.018 #@param {type:"slider", min:0, max:0.02, step:0.0005}
epochs = 110 #@param {type:"slider", min:1, max:150, step:1}

x = tf.Variable(start_x, dtype='float32')  
y = tf.Variable(start_y, dtype='float32')
params = [x, y]
states = []
hyper_params = {"lr": learning_rate}

optimize_banana(SGD_update, params, states, hyper_params)

**Partner Excercise:** Tweak the starting position (x and y), as well as the learning rate and the number of epochs. See if you can get to the global minimum. What do you notice about the behaviour of SGD in the 🍌 valley?

## SGD with Momentum

Hopefully, you've noticed a bit of an issue with SGD, which is that when the gradient is small (i.e. in the 🍌) the changes to the parameters become very small and progress towards the minimum is very slow. One solution to this problem is to add a momentum term to our optimization step:

\begin{align}
\Delta \mathbf{θ} &= γ\Delta \mathbf{θ}+η∇_\mathbf{θ}J(\mathbf{θ}) \\
\mathbf{θ} &= \mathbf{θ}−\Delta \mathbf{θ}
\end{align}

where $\Delta \mathbf{θ}$ is the change in parameters $\mathbf{θ}$ at each time step and is made up of a mixture between the gradients and at given time step and the change from the previous step. $γ$ is called the *momentum* term, and $\eta$ is called the learning rate, as before.

The reason that this method is called *momentum* is that we can compare it to SGD as follows:

> gradient descent is a man walking down a hill. He follows the steepest path downwards; his progress is slow, but steady. Momentum is a heavy ball rolling down the same hill. The added inertia acts both as a smoother and an accelerator, dampening oscillations and causing us to barrel through narrow valleys, small humps and local minima. 

In other words, the momentum term speeds up optimisation if the direction of change stays the same or similar and reduces oscillations when the direction of change goes back and forth. 

We can also describe momentum with some simple pseudo-code:

```
change = 0
for _ in range(epochs):
  grad = calc_grad(loss_func, data, params)
  change = momentum*change + learning_rate*grad
  params = params - change
```





### Implementing Momentum

In [0]:
def momentum_update(params, grads, states, hyper_params):
  # learning_rate=0.01, momentum=0.9
  changes = states['changes']
  for param, grad in zip(params, grads):
    changes[param].assign(hyper_params['momentum']*changes[param] +
                          hyper_params['lr']*grad)
    param.assign_sub(changes[param])

In [0]:
#@title Double click to unhide/hide the code{run: "auto"}
start_x = -0.4 #@param {type:"slider", min:-2, max:2, step:0.1}
start_y = 0.63334 #@param {type:"slider", min:-0.26666, max:1.0666, step:0.1}
lr = 0.001 #@param {type:"slider", min:0, max:0.02,step:0.0005}
momentum = 0.23 #@param {type:"slider", min:0, max:0.99, step:0.01}
epochs = 119 #@param {type:"slider", min:1, max:150,step:1}

x = tf.Variable(start_x, dtype='float32')  
y = tf.Variable(start_y, dtype='float32')
params = [x, y]
changes = {param: tf.Variable(0., dtype='float32') for param in params}
hyper_params = {"lr": learning_rate, "momentum": momentum}
states = {"changes": changes}

optimize_banana(momentum_update, params, states, hyper_params)

**Partner Excercise:** Play around with the various parameters and see if you can get to the minimum. Compare the performance of momentum in the 🍌 to that of SGD.

**Partner Excercise:** Why do we see oscillations for high enough momentum values?

**Partner Excercise:** Do you see any relationship between the amount of momentum and the learning rate?


## RMSProp (Root Mean Square Propagation)

One problem with both Momentum and SGD is that each parameter has the same fixed learning rate. However, we can imagine that:


1.   Each weight might not need to vary by the same amount.
2.   The amount we want to change each parameter changes throughout the optimisation process might change. 

**Excercise:** can you think of examples of when these two cases might apply?

RMSProp is a method that addresses these issues. It can be described with the following formulae:

\begin{align}
\mathbf{v} &= γ\mathbf{v}+(1 - γ)(∇_\mathbf{θ}J(\mathbf{θ}))^2 \\
\mathbf{θ} &= \mathbf{θ}−\frac{η}{\sqrt{\mathbf{v}} + ϵ}∇_\mathbf{θ}J(\mathbf{θ})
\end{align}

where $\mathbf v$ is an estimate of the square of the gradient for each parameter, calculated using a rolling average, $γ$ is a forgetting factor for the rolling average, and $ϵ$ is a small number added for numerical stability. 

In words, RMSProp is scaling down the learning rate for each gradient by rolling average of the most recent gradients for that parameter. Importantly **each parameter has its own learning rate, which changes over time**.

We can also describe RMSPRop using pseudo code:

```
average = 0
for _ in range(epochs):
  grad = calc_grad(loss_func, data, params)
  average = gamma*average + (1 - gamma)*pow(grad, 2)
  params = params - learning_rate/sqrt(average)*grad
 ```

### Implementing RMSProp

In [0]:
def RMSProp_update(params, grads, states, hyper_params):
  # learning_rate=0.001, mixing_prop=0.9, eps=1e-8
  averages = states['averages']
  for param, grad in zip(params, grads):
    averages[param].assign(hyper_params['gamma']*averages[param] +
                           (1 - hyper_params['gamma'])*tf.math.pow(grad, 2))
    param.assign_sub(hyper_params['lr']/(tf.sqrt(averages[param]) + hyper_params['eps'])*grad)

In [0]:
#@title Double click to unhide/hide the code {run: "auto"}
start_x = -0.4 #@param {type:"slider", min:-2, max:2, step:0.1}
start_y = 0.63334 #@param {type:"slider", min:-0.26666, max:1.0666, step:0.1}
lr = 0.012 #@param {type:"slider", min:0, max:0.02,step:0.0005}
gamma = 0.9 #@param {type:"slider", min:0.01, max:0.99, step:0.01}
epochs = 150 #@param {type:"slider", min:1, max:150,step:1}

x = tf.Variable(start_x, dtype='float32')  
y = tf.Variable(start_y, dtype='float32')
params = [x, y]
averages = {param: tf.Variable(0., dtype='float32') for param in params}
states = {"averages": averages}
hyper_params = {"lr": learning_rate, "gamma": gamma, "eps": 1e-8}
  
optimize_banana(RMSProp_update, params, states, hyper_params)

**Partner Excercise:** Play around with the various parameters and see if you can get to the minimum. Compare the performance of RMSProp with momentum and SGD, particularly in the 🍌.

## Adam (Adaptive moment estimation)

Adam combines the ideas of momentum and adaptive learning rates that we have explored above. More specifically, in addition to storing the rolling averages of the *squared* gradients and using them to control the learning rate for each parameter, like RMSProp, it also stores the rolling averages of the gradients themselves and uses them like momentum. Mathematically, we can describe Adam as follows:

\begin{align}
\mathbf m &= β_1\mathbf m+(1−β_1)∇_\mathbf{θ}J(\mathbf{θ}) \\
\mathbf v &= β_2\mathbf v+(1−β_2)(∇_\mathbf{θ}J(\mathbf{θ}))^2 \\
\mathbf{\hat{m}} &= \frac{\mathbf m}{1−β_1^t} \\
\mathbf{\hat{v}} &= \frac{\mathbf v}{1−β_2^t} \\
\mathbf{θ} &= \mathbf{θ}−\frac{η}{\sqrt{\mathbf{\hat{v}}} + ϵ}\mathbf{\hat{m}}
\end{align}

where $\mathbf{m}$ is a rolling estimate of the gradient, $\mathbf{v}$ is a rolling estimate of the squared gradient, and $\mathbf{\hat{m}}$ and $\mathbf{\hat{v}}$ are bias-corrected estimates. As before $η$ and $ϵ$ are the learning rate and a numerical stability term.

The name Adam comes from the fact that $\mathbf{m}$ and $\mathbf{v}$ are estimates of the first moment (the mean) and the second moment (the uncentered variance) of the gradient. The reason that we need bias corrected versions of the estimates is that they are initialized to be zero vectors, which means that when the training starts, the estimates are biased towards zero. This problem is especially relevant when $β_1$ and $β_2$ are close to 1.

The pseudo code for Adam is slightly longer than the other methods we've looked at but should be reasonably simple to understand. If something doesn't make sense, go back and look at momentum and RMSProp.

```
first_moment = 0
second_moment = 0
t = 0
for _ in range(epochs):
  t = t + 1
  grad = calc_grad(loss_func, data, params)
  first_moment = mixing_prop1*first_moment + (1 - beta1)*grad
  second_moment = mixing_prop2*second_moment + (1 - beta2)*pow(grad, 2)
  first_moment_unbiased = first_moment/(1 - pow(beta1, t))
  second_moment_unbiased = second_moment/(1 - pow(beta2, t))
  params = params - learning_rate/sqrt(second_moment_unbiased)*first_moment_unbiased
```

### Implementing Adam

In [0]:
def Adam_update(params, grads, states, hyper_params):
  #learning_rate=0.001, beta1=0.9, beta2=0.999, eps=1e-8
  t = states["t"]
  t.assign_add(1.0)  
  first_moments = states["first_moments"]
  second_moments = states["second_moments"]
  
  for param, grad in zip(params, grads):
    # come back here later!
    # the rest of this function will be removed and participants will implement as an excercise
    first_moments[param].assign(hyper_params["beta1"]*first_moments[param] +
                                (1 - hyper_params["beta1"])*grad)
    second_moments[param].assign(hyper_params["beta2"]*second_moments[param] +
                                 (1 - hyper_params["beta2"])*tf.math.pow(grad, 2))
    
    first_moment_unbiased = first_moments[param]/(1 - tf.pow(beta1, t))
    second_moment_unbiased = second_moments[param]/(1 - tf.pow(beta2, t))
    
    param.assign_sub(hyper_params["lr"]/(tf.sqrt(second_moment_unbiased) +
                                    hyper_params["eps"])*first_moment_unbiased)

In [0]:
#@title Double click to unhide/hide the code {run: "auto"}
start_x = -0.4 #@param {type:"slider", min:-2, max:2, step:0.1}
start_y = 0.63334 #@param {type:"slider", min:-0.26666, max:1.0666, step:0.1}
lr = 0.016 #@param {type:"slider", min:0, max:0.02,step:0.0005}
beta1 = 0.96 #@param {type:"slider", min:0.01, max:0.99, step:0.01}
beta2 = 0.907 #@param {type:"slider", min:0.01, max:0.999, step:0.001}
epochs = 150 #@param {type:"slider", min:1, max:150,step:1}

x = tf.Variable(start_x, dtype='float32')  
y = tf.Variable(start_y, dtype='float32')
params = [x, y]
first_moments = {param: tf.Variable(0., dtype='float32')
                 for param in params}
second_moments = {param: tf.Variable(0., dtype='float32')
                                     for param in params}
t = tf.Variable(0., dtype='float32')
states = {"first_moments": first_moments, 
          "second_moments": second_moments, "t": t}
hyper_params = {"lr": learning_rate, "beta1": beta1, "beta2": beta2, "eps": 1e-8}

optimize_banana(Adam_update, params, states, hyper_params)

## Learning Rate Decay

One of the advantages of methods such as Adam and RMSProp is that the effective learning rate for each parameter is no longer a constant during training. This behaviour is desirable because we often want to reduce the learning rate as we near the global minimum so that we do not shoot past it. Another simple strategy for solving this problem is *learning rate decay*. With learning rate decay, we progressively reduce the learning rate during the training process. For example, we might decay our learning rate as follows:

\begin{equation}
  η = η \times \frac{1}{1 + \delta \times t}
\end{equation}

where $η$ is the learning rate, $\delta$ is the decay rate, and $t$ is the number of the current training epoch. This method will give us a learning rate that looks something like this:

In [0]:
η = 0.01
epochs = 100
δ = η/epochs
ηs = [η]
for t in range(epochs):
  ηs.append(ηs[t]*1/(1 + δ*t))
  
plt.plot(ηs)
plt.show()

In practice, while we do often use the simple decay scheme shown above, it is also common to use more complicated *learning rate schedules* such as exponential decay and step decay:

In [0]:
# exponential decay
η = 0.01
epochs = 100
δ = 0.01
ηs = []
for t in range(epochs):
  ηs.append(η*np.e**(-δ*t))
  
plt.plot(ηs)
plt.show()

In [0]:
# step decay
η = 0.01
epochs = 100
epochs_wait = 10
δ = 0.5
ηs = []
for t in range(epochs):
  ηs.append(η*δ**np.floor((1+t)/epochs_wait))
  
plt.plot(ηs)
plt.show()

Any of these learning rate decay methods are compatible with the optimisation methods described above. The choices of whether or not to use learning rate decay and if so, which method to use are hyper-parameters that we can tune.


## Putting it all into practice

TODO:

* practical advice

Putting all of this into practice is very simple! Keras provides us with a high-level API that makes using any of the optimization methods or learning rate schedules as easy as changing a single line of code. Of course, if you are defining a custom training loop using `tf.GradientTape` then the code we've used above can easily be converted to work for any model and dataset. 

One of the advantages of using Keras is that it provides us with reasonable default values for all of the hyper-parameters of the optimization algorithms. 

Let's use Keras to train a simple MLP on FashionMNIST so that we can compare the optimization algorithms we've looked at in a more realistic setting.

As a quick reminder, FashionMNIST contains 28x28 grayscale images from 10 different types of clothing. Let's take a quick look:

In [0]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

text_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

    img_index = np.random.randint(0, 50000)
    plt.imshow(train_images[img_index], cmap="gray_r")
    plt.xlabel(text_labels[train_labels[img_index]])

Before we train on the data, we want to do a little pre-processing. We won't go into detail about how and why we are doing the pre-processing, but if you want to know more, you should take a look at the *Deep Feedforward Networks* practical.

In [0]:
batch_size = 128

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
# Divide image values and cast to float so that they end up as a floating point number between 0 and 1
train_ds = train_ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))
# Shuffle the examples.
train_ds = train_ds.shuffle(buffer_size=batch_size * 10)
# Now "chunk" the examples into batches
train_ds = train_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_ds = test_ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))
test_ds = test_ds.batch(batch_size)

Now let's define a simple MLP:

In [0]:
def build_mlp():
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
  ])
  
  return model

model = build_mlp()

model.summary()

And finally, lets train the MLP using a few different optimizers and compare the results:

In [0]:
#@title Helper functions (double click to unhide/hide the code)

def make_loss_plots(losses):
  plt.close()
  for label, loss_vals in losses.items():
    plt.plot(loss_vals, label=label)
  plt.legend()
  plt.show()

In [0]:
losses = {}

# SGD
model = build_mlp()
model.compile(optimizer='sgd', 
              # we can use the string shortcut if we 
              # don't want to change the hyper-parameters
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
sgd_hist = model.fit(train_ds, epochs=5,
          validation_data=test_ds)

losses['SGD'] = sgd_hist.history["val_loss"]
make_loss_plots(losses)



# Momentum
model = build_mlp()
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.0),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
momentum_hist = model.fit(train_ds, epochs=5,
          validation_data=test_ds)

losses['Momentum'] = momentum_hist.history["val_loss"]
make_loss_plots(losses)

# RMSProp
model = build_mlp()
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9),
              # rho is the symbol used for the forget factor in Keras
              # we used gamma (γ) in this practical
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
rmsprop_hist = model.fit(train_ds, epochs=5,
          validation_data=test_ds)

losses['RMSProp'] = rmsprop_hist.history["val_loss"]
make_loss_plots(losses)

# Adam
model = build_mlp()
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
adam_hist = model.fit(train_ds, epochs=5,
          validation_data=test_ds)

losses['Adam'] = adam_hist.history["val_loss"]
make_loss_plots(losses)

# SGD with learning rate decay
model = build_mlp()
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.05, momentum=0.0, decay=0.9),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
sgd_decay_hist = model.fit(train_ds, epochs=5,
          validation_data=test_ds)

losses['SGD with decay'] = sgd_decay_hist.history["val_loss"]
make_loss_plots(losses)

**Exercise:** Play with the parameters of the various optimizers and see how that affects this particular problem.

**Note:** There is an element of randomness each time we train our models; this means that we should be careful when making judgements about which methods are best for this problem.

## Optional extra reading: second order methods

TODO: discus methods like BFGS

## Optional extra reading: weight decay vs L2 norm regularization

TODO: weight decay is a way of implementing L2 norm regularization in the optimizer

TODO: discuss AdamW?

## Conclusion: what method should you use?

There are no hard rules for which methods you should use. It will always depend on your particular model and dataset. However, some guidelines are worth keeping in mind:

1. Adam typically works very well in a large number of settings and is usually a good first choice.
2. RMSProp can often outperform Adam for RNNs as well as in RL. If you are working in either of these domains, then it might be worth trying RMSProp.
3. Good old SGD and SGD with momentum often work just as well as more sophisticated methods like Adam. Don't think that they aren't worth trying out just because they are simple.




## Tasks

1. **[Optional, Advanced]** Implement Nesterov Momentum. You can read more about it [here](http://ruder.io/optimizing-gradient-descent/).
2. **[All]** Combine the implementations for momentum and RMSProp to implement Adam. Experiment with the parameters of Adam and compare it to the previous methods we have looked at.
3. **[Optional, Advanced]** Implement Nadam. You can read more about it [here](http://ruder.io/optimizing-gradient-descent/).
4. **[All]** Augment any of the optimization methods with learning rate decay. You can choose any of the learning rate decay methods. Play around with the parameters. Compare the performance of the optimization method with and without learning rate decay.
5. **[Intermediate]** Implement the other two learning rate decay methods and compare all three decay methods with one another.




## Extra resources

* A [blog post](http://fa.bianp.net/teaching/2018/eecs227at/gradient_descent.html) from Fabian Pedregosa on gradient descent [**Highly Recommended**].
* [Distil.pub post](https://distill.pub/2017/momentum/) by Gabriel Goh on why momentum works [**Highly Recommended**].
* [Sebastian Ruder's blog](http://ruder.io/optimizing-gradient-descent/) on gradient descent algorithms.
* Deep Dive into Deep Learning chapter on [Optimization Algorithms](http://d2l.ai/chapter_optimization/index.html).
* Keras optimizer [docs](https://keras.io/optimizers/).
